# Scrapping - Github Repositories 
- 수집 데이터 수 : 키워드당 100 개
- 타겟 : 페이지당 10개의 데이터를 조회할 수 있고 총 10페이지를 순회합니다.

평점을 받지 못한 강의가 있는 페이지는 건너뜁니다. 이 때 10개의 데이터가 일괄 누락되지만,
별점 순으로 검색하므로 누락 된다면 해당 페이지는 무의미한 값으로 판단해도 됩니다.

-----

### udemy_scrapper() 동작 순서
1. selenium webdrive를 통해 Github 사이트에 접속합니다.
2. Input 값으로 입력한 Keyword를 검색합니다.
4. url 값을 통해 **stars** 순으로 내림차순 정렬합니다.
5. 총 10페이지를 순회하며 데이터를 순차적으로 저장합니다.
6. * 크롤러 탐지를 피하기 위해 페이지마다 0.6초씩 대기해줍니다.
7-1. 각 star 데이터는 '0.0k' 형태의 문자열로 표기되어 있습니다. 'k'는 '000'으로 변환하면서 수집합니다.
7-2. list comprehension을 통해 '.'이 있다면 없애고 0을 한개 빼줍니다. 마지막으로 star는 int형으로 변환합니다
8. 전역변수로 데이터프레임을 생성합니다.

In [125]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
import pandas as pd
import time

## Test Code

In [42]:
# 크롬 드라이버 할당, 페이지 열기
drive = webdriver.Chrome(executable_path='../chromedriver')
link = "https://github.com/search?p={}&q={}&type=repositories".format(1,'java')
drive.get(link)

In [43]:
link_li = drive.find_elements_by_css_selector('div.f4 > a.v-align-middle')

In [50]:
for link in link_li:
    print(link.get_attribute('href'))

https://github.com/TheAlgorithms/Java
https://github.com/DuGuQiuBai/Java
https://github.com/Snailclimb/JavaGuide
https://github.com/hmkcode/Java
https://github.com/HelloWorld521/Java
https://github.com/kubernetes-client/java
https://github.com/gaopu/Java
https://github.com/pubnub/java
https://github.com/singgel/JAVA
https://github.com/json-iterator/java


In [70]:
star_li = drive.find_elements_by_css_selector('div.d-flex > div.mr-3 > a.muted-link')

In [71]:
star_li[1].text

'7.1k'

In [79]:
for i in star_li:
    print(i)

<selenium.webdriver.remote.webelement.WebElement (session="73e743579a5ff4138873e85fe787edf7", element="65086e71-ebc0-4195-9e46-f5647c617dda")>
<selenium.webdriver.remote.webelement.WebElement (session="73e743579a5ff4138873e85fe787edf7", element="3b936964-09f3-4e03-8e34-b2a71909fe69")>
<selenium.webdriver.remote.webelement.WebElement (session="73e743579a5ff4138873e85fe787edf7", element="dc6e2bd0-6158-4e35-b36b-3479e8028113")>
<selenium.webdriver.remote.webelement.WebElement (session="73e743579a5ff4138873e85fe787edf7", element="323b03c3-dd30-41d0-9ef2-02f918cb20de")>
<selenium.webdriver.remote.webelement.WebElement (session="73e743579a5ff4138873e85fe787edf7", element="7b3f63c0-0854-4c93-aa02-6d65cc9e495b")>
<selenium.webdriver.remote.webelement.WebElement (session="73e743579a5ff4138873e85fe787edf7", element="c8394e3b-601f-4bec-87b8-dae94520b18d")>
<selenium.webdriver.remote.webelement.WebElement (session="73e743579a5ff4138873e85fe787edf7", element="2134da75-0dd9-400c-8a7b-3876bb10bba8")>

In [108]:
def counts(x):
    if '.' in x:
        star = x.replace('.', '')
        star = star[:-1]
    return int(star)

In [91]:
star_li

,0
0,<selenium.webdriver.remote.webelement.WebEleme...
1,<selenium.webdriver.remote.webelement.WebEleme...
2,<selenium.webdriver.remote.webelement.WebEleme...
3,<selenium.webdriver.remote.webelement.WebEleme...
4,<selenium.webdriver.remote.webelement.WebEleme...
5,<selenium.webdriver.remote.webelement.WebEleme...
6,<selenium.webdriver.remote.webelement.WebEleme...
7,<selenium.webdriver.remote.webelement.WebEleme...
8,<selenium.webdriver.remote.webelement.WebEleme...
9,<selenium.webdriver.remote.webelement.WebEleme...


### Github - 저장소 명, 저장소 설명, 별점, 저장소 링크 수집

In [134]:
def github_scrapper(keyword):
    drive = webdriver.Chrome(executable_path='../chromedriver')
    title = []
    summary = []
    star = []
    link = []
    for page in range(1,5):
        site = 'https://github.com/search?o=desc&p={}&q={}&s=stars&type=Repositories'.format(page,keyword)
        drive.get(site)
        time.sleep(0.6)
        title_li = drive.find_elements_by_css_selector('div.f4 > a.v-align-middle')
        summary_li = drive.find_elements_by_css_selector('div.mt-n1 > p.mb-1')
        star_li = drive.find_elements_by_css_selector('div.d-flex > div.mr-3 > a.muted-link')
        print('데이터를 수집하고 있습니다. - 진행률 : {}/4 '.format(page))
        if len(title_li) == len(star_li):
            for i in range(len(title_li)):
                title.append(title_li[i].text)
                summary.append(summary_li[i].text)
                star.append(star_li[i].text.replace('k','000'))
                link.append(title_li[i].get_attribute('href'))
        else:
            print('별점이 부족합니다.')
            continue
    star = [int(i[:-1].replace('.','')) if '.' in i else int(i) for i in star]
    global github_df
    github_df = pd.DataFrame(list(zip(title,summary,star,link)), columns = ['Title', 'Summary', 'Stars', 'Link'])
    drive.quit()

## 사용 예시 1 : Keyword - 'Java'

In [135]:
github_scrapper('Java')

데이터를 수집하고 있습니다. - 진행률 : 1/4 
데이터를 수집하고 있습니다. - 진행률 : 2/4 
데이터를 수집하고 있습니다. - 진행률 : 3/4 
데이터를 수집하고 있습니다. - 진행률 : 4/4 


In [131]:
github_df

,Title,Summary,Stars,Link
0,CyC2018/CS-Notes,📚 技术面试必备基础知识、Leetcode、计算机操作系统、计算机网络、系统设计、Java、...,112000,https://github.com/CyC2018/CS-Notes
1,airbnb/javascript,JavaScript Style Guide,99900,https://github.com/airbnb/javascript
2,Snailclimb/JavaGuide,「Java学习+面试指南」一份涵盖大部分Java程序员所需要掌握的核心知识。准备 Java ...,88500,https://github.com/Snailclimb/JavaGuide
3,trekhleb/javascript-algorithms,📝 Algorithms and data structures implemented i...,81400,https://github.com/trekhleb/javascript-algorithms
4,iluwatar/java-design-patterns,Design patterns implemented in Java,60800,https://github.com/iluwatar/java-design-patterns
5,elastic/elasticsearch,"Open Source, Distributed, RESTful Search Engine",51300,https://github.com/elastic/elasticsearch
6,spring-projects/spring-boot,Spring Boot,50700,https://github.com/spring-projects/spring-boot
7,kdn251/interviews,Everything you need to know to get the job.,47800,https://github.com/kdn251/interviews
8,doocs/advanced-java,😮 互联网 Java 工程师进阶知识完全扫盲：涵盖高并发、分布式、高可用、微服务、海量数据处...,47500,https://github.com/doocs/advanced-java
9,ReactiveX/RxJava,RxJava – Reactive Extensions for the JVM – a l...,43500,https://github.com/ReactiveX/RxJava
